In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re

In [5]:
df = pd.DataFrame({'Mark': pd.Series(dtype='str'),
                   'Model': pd.Series(dtype='str'),
                   'Price': pd.Series(dtype='int'),
                   'Release_Year': pd.Series(dtype='int'),
                   'Mileage': pd.Series(dtype='int'),
                   'Owners': pd.Series(dtype='str'),
                   'Status': pd.Series(dtype='str'),
                   'Power': pd.Series(dtype='int'),
                   'Engine_capacity': pd.Series(dtype='float'),
                   'Engine_type': pd.Series(dtype='str'),
                   'Transmission': pd.Series(dtype='str'),
                   'Drive': pd.Series(dtype='str'),
                   'Body_type': pd.Series(dtype='str'),
                   'Color': pd.Series(dtype='str')})


In [18]:
class Parser:
    def __init__(self):
        self.driver = webdriver.Chrome()
        self.navigate()

    def navigate(self):
        global df
        self.driver.get('https://www.avito.ru/kazan/avtomobili/s_probegom-ASgBAgICAUSGFMjmAQ?cd=1&radius=200')

        href = []

        limit_href = 19000
        count_href = 0
        page_number = 1
        kazan = False
        spb = False
        moscow = False
        nn = False
        kazan_word = 'kazan'
        spb_word = 'sankt-peterburg'
        moscow_word = 'moskva'
        nn_word = 'nizhniy_novgorod'
        city_word = kazan_word
        while 1 == 1 and count_href < limit_href:
        
            if page_number == 101:
                if kazan is False:
                    kazan = True
                    page_number = 1
                    city_word = spb_word
                elif spb is False: 
                    spb = True
                    page_number = 1
                    city_word = moscow_word
                elif moscow is False:
                    moscow = True
                    page_number = 1
                    city_word = nn_word
                else:
                    nn = True
                    page_number = 1
                    print("No more links")
                    break
                
            link = f'https://www.avito.ru/{city_word}/avtomobili/s_probegom-ASgBAgICAUSGFMjmAQ?cd=1&p={page_number}&radius=200'
                
            self.driver.get(link)

            try:
                # Находим все машины на текущей странице и записываем ссылки в массив
                items = self.driver.find_elements(by=By.XPATH,
                                                  value='//a[@class="link-link-MbQDP link-design-default-_nSbv title-root-zZCwT iva-item-title-py3i_ title-listRedesign-_rejR title-root_maxHeight-X6PsH"]')
                for i in range(len(items)):
                    href.append(items[i].get_attribute('href'))

                count_href += len(items)
                if count_href > limit_href:
                    break
                page_number += 1
                print(f'Всего собрано {count_href } ссылок')
            except:
                print("There is no links")
                break


        print('Всего найдено автомобилей:' + str(len(href)))

        # цикл по записанным страницам для записи данных о машине
        for i in range(len(href)):
            try:
                self.driver.get(href[i])
                # выводим ссылку
                print(f"{i}: {href[i]}")
            except:
                continue

            # выводим название марки и модели
            try:
                spans = self.driver.find_elements(by=By.XPATH, value='//a[@class="breadcrumbs-link-Vr4Nc"]')
                model = spans[-2].text
                mark = spans[-3].text
                
                title_span = self.driver.find_element(by=By.XPATH, value='//span[@class="title-info-title-text"]').text
                
                generation = spans[-1].text
                # если в навигации нет ссылки на тип кузова, то в generation записалось название модели
                if title_span.find(generation) != -1:
                    mark = spans[-2].text
                    model = spans[-1].text
                print(mark + ' ' + model)
            except:
                print("Mark or model Not found")
                continue

            # выводим год выпуска
            try:
                li = self.driver.find_element(by=By.XPATH, value='//span[text() = "Год выпуска"]//ancestor::li').text
                release_year = li.split(' ')[-1]
                release_year = int(release_year) if release_year.isnumeric() else None
                # print(f"Год выпуска: {release_year}")
            except:
                print("Release Year Not found")
                continue

            # выводим стоимость автомобиля
            try:
                span = self.driver.find_element(by=By.XPATH, value='//span[@class="js-item-price style-item-price-text-_w822 text-text-LurtD text-size-xxl-UPhmI"]').text
                price = span.replace(' ', '')
                price = int(price) if price.isnumeric() else None
                # print(f"Стоимость: {price}")
            except:
                print("Price not found")
                continue

            # выводим пробег
            try:
                li = self.driver.find_element(by=By.XPATH, value='//span[text() = "Пробег"]//ancestor::li').text
                # -2, чтобы взять только число без км
                mileage = li.split(' ')[-2]
                mileage = int(mileage) if mileage.isnumeric() else None
                # print(f"Пробег: {mileage} км")
            except:
                print("Пробег: None")
                mileage = None

            # выводим Объём двигателя
            try:
                li = self.driver.find_element(by=By.XPATH, value='//span[text() = "Объём двигателя"]//ancestor::li')\
                    .text
                engine_capacity = li.split(' ')[-2]
                engine_capacity = float(engine_capacity) if re.match(r'^-?\d+(?:\.\d+)$', engine_capacity) is not None \
                    else None
                # print(engine_capacity)
            except:
                print("Объём двигателя: None")
                engine_capacity = None

            # выводим Тип двигателя
            try:
                li = self.driver.find_element(by=By.XPATH, value='//span[text() = "Тип двигателя"]//ancestor::li').text
                engine_type = li.split(' ')[-1]
                # print(engine_type)
            except:
                print("Тип двигателя: None")
                engine_type = None

            # выводим Коробка передач
            try:
                li = self.driver.find_element(by=By.XPATH, value='//span[text() = "Коробка передач"]//ancestor::li').text
                transmission = li.split(' ')[-1]
                # print(transmission)
            except:
                print("Коробка передач: None")
                transmission = None

            # выводим Привод
            try:
                li = self.driver.find_element(by=By.XPATH, value='//span[text() = "Привод"]//ancestor::li').text
                drive = li.split(' ')[-1]
                # print(drive)
            except:
                print("Привод: None")
                drive = None

            # выводим Тип кузова
            try:
                li = self.driver.find_element(by=By.XPATH, value='//span[text() = "Тип кузова"]//ancestor::li').text
                body_type = li.split(' ')[-1]
                # print(body_type)
            except:
                print("Тип кузова: None")
                body_type = None

            # выводим Состояние
            try:
                li = self.driver.find_element(by=By.XPATH, value='//span[text() = "Состояние"]//ancestor::li').text
                status = li.replace('Состояние: ', '')
                # print(status)
            except:
                print("Состояние: None")
                status = None

            # выводим Владельцев по ПТС
            try:
                li = self.driver.find_element(by=By.XPATH, value='//span[text() = "Владельцев по ПТС"]//ancestor::li').text
                owners = li.replace('Владельцев по ПТС: ', '')
                # print(owners)
            except:
                print("Владельцев по ПТС: None")
                owners = None

            # выводим Цвет
            try:
                li = self.driver.find_element(by=By.XPATH, value='//span[text() = "Цвет"]//ancestor::li').text
                color = li.split(' ')[-1]
                # print(color)
            except:
                print("Цвет: None")
                color = None

            # выводим Модификация
            try:
                li = self.driver.find_element(by=By.XPATH, value='//span[text() = "Модификация"]//ancestor::li')
                list_modification = li.text.split(' ')
                power_engine = list_modification[-2].replace('(', '').replace(')', '')
                power_engine = int(power_engine) if power_engine.isnumeric() else None
                # print(f"Мощность двигателя: {power_engine}")
            except:
                print("Модификация: None")
                power_engine = None

            df = df.append({'Mark': mark, 'Model': model, 'Release_Year': release_year,
                                    'Mileage': mileage, 'Owners': owners, 'Price': price,
                                    'Status': status, 'Power': power_engine, 'Engine_capacity': engine_capacity,
                                    'Engine_type': engine_type, 'Transmission': transmission, 'Drive': drive,
                                    'Body_type': body_type, 'Color': color}, ignore_index=True)


In [19]:
b = Parser()
print(f"{len(df.index)} entries was parsed")
print(df.head())
df.info()
df.to_excel('parsed_car_data.xlsx')

Всего собрано 50 ссылок
Всего собрано 100 ссылок
Всего собрано 150 ссылок
Всего собрано 200 ссылок
Всего собрано 250 ссылок
Всего собрано 300 ссылок
Всего собрано 350 ссылок
Всего собрано 400 ссылок
Всего собрано 450 ссылок
Всего собрано 500 ссылок
Всего собрано 550 ссылок
Всего собрано 600 ссылок
Всего собрано 650 ссылок
Всего собрано 700 ссылок
Всего собрано 750 ссылок
Всего собрано 800 ссылок
Всего собрано 850 ссылок
Всего собрано 900 ссылок
Всего собрано 950 ссылок
Всего собрано 1000 ссылок
Всего собрано 1050 ссылок
Всего собрано 1100 ссылок
Всего собрано 1150 ссылок
Всего собрано 1200 ссылок
Всего собрано 1250 ссылок
Всего собрано 1300 ссылок
Всего собрано 1350 ссылок
Всего собрано 1400 ссылок
Всего собрано 1450 ссылок
Всего собрано 1500 ссылок
Всего собрано 1550 ссылок
Всего собрано 1600 ссылок
Всего собрано 1650 ссылок
Всего собрано 1700 ссылок
Всего собрано 1750 ссылок
Всего собрано 1800 ссылок
Всего собрано 1850 ссылок
Всего собрано 1900 ссылок
Всего собрано 1950 ссылок
Всего 

Всего собрано 15648 ссылок
Всего собрано 15698 ссылок
Всего собрано 15748 ссылок
Всего собрано 15798 ссылок
Всего собрано 15848 ссылок
Всего собрано 15898 ссылок
Всего собрано 15948 ссылок
Всего собрано 15998 ссылок
Всего собрано 16048 ссылок
Всего собрано 16098 ссылок
Всего собрано 16148 ссылок
Всего собрано 16198 ссылок
Всего собрано 16248 ссылок
Всего собрано 16298 ссылок
Всего собрано 16348 ссылок
Всего собрано 16398 ссылок
Всего собрано 16448 ссылок
Всего собрано 16498 ссылок
Всего собрано 16548 ссылок
Всего собрано 16598 ссылок
Всего собрано 16648 ссылок
Всего собрано 16698 ссылок
Всего собрано 16748 ссылок
Всего собрано 16798 ссылок
Всего собрано 16848 ссылок
Всего собрано 16898 ссылок
Всего собрано 16948 ссылок
Всего собрано 16998 ссылок
Всего собрано 17048 ссылок
Всего собрано 17098 ссылок
Всего собрано 17148 ссылок
Всего собрано 17198 ссылок
Всего собрано 17248 ссылок
Всего собрано 17298 ссылок
Всего собрано 17348 ссылок
Всего собрано 17398 ссылок
Всего собрано 17448 ссылок
В

Hyundai Solaris
71: https://www.avito.ru/ulyanovsk/avtomobili/lada_kalina_2013_2636973906
ВАЗ (LADA) Kalina
72: https://www.avito.ru/cheboksary/avtomobili/nissan_primera_2007_2597182888
Nissan Primera
73: https://www.avito.ru/zelenodolsk/avtomobili/mazda_6_2007_2631390188
Mazda 6
74: https://www.avito.ru/kazan/avtomobili/vaz_lada_granta_2013_2526125828
ВАЗ (LADA) Granta
75: https://www.avito.ru/nizhnekamsk/avtomobili/ford_kuga_2013_2607207096
Ford Kuga
76: https://www.avito.ru/cheboksary/avtomobili/volkswagen_caddy_2019_2620482807
Volkswagen Caddy
77: https://www.avito.ru/kazan/avtomobili/volkswagen_polo_2020_2567384693
Volkswagen Polo
78: https://www.avito.ru/cheboksary/avtomobili/volkswagen_polo_2011_2651707623
Volkswagen Polo
79: https://www.avito.ru/kamskoe_uste/avtomobili/nissan_kix_2011_2581994603
Nissan Kix
80: https://www.avito.ru/kazan/avtomobili/volkswagen_polo_2016_2690327007
Volkswagen Polo
81: https://www.avito.ru/kazan/avtomobili/mazda_3_2009_2629877917
Mazda 3
82: https:

Hyundai Sonata
161: https://www.avito.ru/cheboksary/avtomobili/vaz_lada_priora_2011_2594963426
ВАЗ (LADA) Priora
162: https://www.avito.ru/ulyanovsk/avtomobili/kia_ceed_2011_2531855404
Kia Ceed
163: https://www.avito.ru/ulyanovsk/avtomobili/honda_civic_2009_2647106560
Honda Civic
164: https://www.avito.ru/yoshkar-ola/avtomobili/volkswagen_polo_2012_2541820772
Volkswagen Polo
165: https://www.avito.ru/moskva/avtomobili/ford_transit_2007_2630462155
Ford Transit
166: https://www.avito.ru/cheboksary/avtomobili/volkswagen_passat_2012_2674659632
Volkswagen Passat
167: https://www.avito.ru/ulyanovsk/avtomobili/honda_civic_2010_2646557358
Honda Civic
168: https://www.avito.ru/kazan/avtomobili/mitsubishi_asx_2010_2644037910
Mitsubishi ASX
169: https://www.avito.ru/cheboksary/avtomobili/nissan_sentra_2015_2623533087
Nissan Sentra
170: https://www.avito.ru/cheboksary/avtomobili/skoda_rapid_2018_2499557466
Skoda Rapid
171: https://www.avito.ru/ulyanovsk/avtomobili/kia_sportage_2007_2603586750
Kia 

Hyundai Sonata
253: https://www.avito.ru/nizhnekamsk/avtomobili/kia_sportage_2015_2639859698
Kia Sportage
254: https://www.avito.ru/ulyanovsk/avtomobili/toyota_mark_ii_1993_2690552396
Toyota Mark II
255: https://www.avito.ru/kazan/avtomobili/mazda_6_2011_2570884284
Mazda 6
256: https://www.avito.ru/ulyanovsk/avtomobili/nissan_almera_2013_2607745019
Nissan Almera
257: https://www.avito.ru/sernur/avtomobili/skoda_octavia_2008_2676768783
Skoda Octavia
258: https://www.avito.ru/nizhnekamsk/avtomobili/honda_mobilio_2002_2260474899
Honda Mobilio
259: https://www.avito.ru/novocheboksarsk/avtomobili/skoda_octavia_2011_2621553656
Skoda Octavia
260: https://www.avito.ru/kazan/avtomobili/citroen_c4_2014_2586826427
Citroen C4
261: https://www.avito.ru/kazan/avtomobili/mercedes-benz_e-klass_2012_2548824588
Mercedes-Benz E-класс
262: https://www.avito.ru/ulyanovsk/avtomobili/toyota_camry_2012_2622945519
Toyota Camry
263: https://www.avito.ru/cheboksary/avtomobili/chevrolet_lacetti_2005_2466353851
Ch

...

17410: https://www.avito.ru/vladivostok/avtomobili/kia_k3_2019_2557841622
Kia K3
17411: https://www.avito.ru/melenki/avtomobili/vaz_lada_1111_oka_2008_2656182877
ВАЗ (LADA) 1111 Ока
17412: https://www.avito.ru/dzerzhinsk/avtomobili/hyundai_santa_fe_2019_2590410500
Hyundai Santa Fe
17413: https://www.avito.ru/moskva/avtomobili/toyota_highlander_2012_2539092884
Toyota Highlander
17414: https://www.avito.ru/moskva/avtomobili/bmw_3_seriya_2015_2565607219
BMW 3 серия
17415: https://www.avito.ru/kovrov/avtomobili/bmw_x5_2018_2554435394
BMW X5
17416: https://www.avito.ru/nizhniy_novgorod/avtomobili/mazda_premacy_2016_2613205100
Mazda Premacy
17417: https://www.avito.ru/cheboksary/avtomobili/volkswagen_polo_2017_2611397225
Volkswagen Polo
17418: https://www.avito.ru/murom/avtomobili/toyota_avensis_2009_2304721225
Toyota Avensis
17419: https://www.avito.ru/moskva/avtomobili/volkswagen_polo_2021_2619755067
Volkswagen Polo
17420: https://www.avito.ru/moskva/avtomobili/kia_sorento_2012_2607003959


17496: https://www.avito.ru/moskva/avtomobili/bmw_x6_2010_2515974044
BMW X6
17497: https://www.avito.ru/nizhniy_novgorod/avtomobili/kia_ceed_2010_2648575346
Kia Ceed
17498: https://www.avito.ru/arzamas/avtomobili/volkswagen_golf_plus_2013_2611663826
Volkswagen Golf Plus
17499: https://www.avito.ru/moskva/avtomobili/kia_soul_2016_2676098147
Kia Soul
17500: https://www.avito.ru/vladivostok/avtomobili/nissan_x-trail_2019_2595703040
Nissan X-Trail
17501: https://www.avito.ru/dzerzhinsk/avtomobili/nissan_skyline_1992_2508178658
Nissan Skyline
17502: https://www.avito.ru/murom/avtomobili/vaz_lada_granta_2012_2551301643
ВАЗ (LADA) Granta
17503: https://www.avito.ru/kazan/avtomobili/ford_focus_2017_2414105565
Ford Focus
17504: https://www.avito.ru/nizhniy_novgorod/avtomobili/vaz_lada_granta_2021_2474440931
ВАЗ (LADA) Granta
17505: https://www.avito.ru/nizhniy_novgorod/avtomobili/datsun_mi-do_2015_2672293651
Datsun mi-DO
17506: https://www.avito.ru/naberezhnye_chelny/avtomobili/subaru_forester_

Release Year Not found
17582: https://www.avito.ru/dzerzhinsk/avtomobili/opel_astra_gtc_2012_2536204941
Opel Astra GTC
17583: https://www.avito.ru/kameshkovo/avtomobili/jeep_grand_cherokee_2006_2620411608
Jeep Grand Cherokee
17584: https://www.avito.ru/ekaterinburg/avtomobili/toyota_avensis_2008_2531994842
Toyota Avensis
17585: https://www.avito.ru/moskva/avtomobili/mercedes-benz_a-klass_2013_2565366348
Mercedes-Benz A-класс
17586: https://www.avito.ru/moskva/avtomobili/hyundai_ix35_2014_2539697203
Hyundai ix35
17587: https://www.avito.ru/nizhniy_novgorod/avtomobili/vaz_lada_2115_samara_2007_2600627243
ВАЗ (LADA) 2115 Samara
17588: https://www.avito.ru/moskva/avtomobili/ford_focus_2013_2630135830
Ford Focus
17589: https://www.avito.ru/kudrovo/avtomobili/mercedes-benz_cla-klass_2019_2523967219
Mercedes-Benz CLA-класс
17590: https://www.avito.ru/gidrotorf/avtomobili/kia_soul_2020_2617351036
Kia Soul
17591: https://www.avito.ru/sankt-peterburg/avtomobili/land_rover_discovery_sport_2017_26

ВАЗ (LADA) Granta
17666: https://www.avito.ru/arzamas/avtomobili/vaz_lada_2107_2006_2605289491
ВАЗ (LADA) 2107
17667: https://www.avito.ru/moskva/avtomobili/audi_rs5_2010_2499454176
Audi RS5
17668: https://www.avito.ru/nizhniy_novgorod/avtomobili/vaz_niva_travel_2021_2650428983
ВАЗ (LADA) Niva Travel
17669: https://www.avito.ru/bor/avtomobili/hyundai_grand_starex_2008_2559993350
Hyundai Grand Starex
17670: https://www.avito.ru/tsentralnyy/avtomobili/land_rover_range_rover_2008_2545615667
Land Rover Range Rover
17671: https://www.avito.ru/pamyat_parizhskoy_kommuny/avtomobili/vaz_lada_2112_2002_2597351406
ВАЗ (LADA) 2112
17672: https://www.avito.ru/kineshma/avtomobili/kia_sorento_2012_2640032807
Kia Sorento
17673: https://www.avito.ru/nizhniy_novgorod/avtomobili/vaz_lada_21099_2001_2676402278
ВАЗ (LADA) 21099
17674: https://www.avito.ru/ivanovo/avtomobili/mitsubishi_lancer_2005_2623343004
Mitsubishi Lancer
17675: https://www.avito.ru/vladivostok/avtomobili/fiat_500_2019_2595255896
FIAT 5

17752: https://www.avito.ru/cheboksary/avtomobili/vaz_lada_2111_2006_2638261015
ВАЗ (LADA) 2111
17753: https://www.avito.ru/moskva/avtomobili/hyundai_grand_santa_fe_2014_2271297970
Hyundai Grand Santa Fe
17754: https://www.avito.ru/moskva/avtomobili/bmw_6_seriya_gt_2018_2467468597
BMW 6 серия GT
17755: https://www.avito.ru/nizhniy_novgorod/avtomobili/kia_rio_2010_2487642298
Mark or model Not found
17527 entries was parsed
         Mark        Model    Price  Release_Year Mileage Owners    Status  \
0  ВАЗ (LADA)         2110    90000          2001  255000     4+  Не битый   
1      Nissan       Almera   563500          2014  170000      1  Не битый   
2         Kia         Ceed  1800000          2022    2670      1  Не битый   
3     Hyundai       Staria  3999000          2022    5000      1  Не битый   
4  ВАЗ (LADA)  2114 Samara    79000          2007  164000      3  Не битый   

  Power Engine_capacity Engine_type Transmission     Drive Body_type  \
0    94             1.5      Бенз